# Linear Algebra Basics

### a.k.a. "Stuff You've Seen Before"

Assuming you've drunk the Kool-Aid that linear algebra is useful, let's start dusting off some of those neurons from your introductory classes. In this notebook, I'll focus on visualizations and applications for these topics:

* Rank
* Nullspace
* Determinant & Singularity


### Python Setup

In [1]:
# Render MPL figures within notebook cells
%matplotlib inline

# Import python libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [2]:
# Configure some defaults for plots
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (10, 3)

In [3]:
# Set Numpy's random number generator so the same results are produced each time the notebook is run
np.random.seed(0)

### The Sensor Interpretation of a Linear System

It can be helpful to imagine $y=Ax$ as the following:

* $x$ describes the outputs of $n$ transmitters
* $y$ contains $m$ sensor measurements
* $a_{i,j}$ is the impact of transmitter $j$ on sensor $i$

<img src='img/sensor_interp.png' style='height: 200px'>

With this picture, you can start to think of columns as the "fingerprints" of the inputs - if you crank up the transmission power from $x_3$, then $y$ starts to look more like $a_3$. Another perspective is that the vector $x$ is a recipe for mixture of columns of $A$.

### Rank

The **rank** of a matrix is a number that tells you the size of the largest group of rows or columns of $A$ that form a [linearly independent](https://en.wikipedia.org/wiki/Linear_independence) set.

Consider the following matrix:

In [4]:
# Create a matrix whose rows form a linearly-independent set
A = np.eye(3)
A

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

No matter how hard I want to, there is no way that I can mix the first two rows of $A$ by scaling and adding them that will result in the third row. Thus, the rank of $A$ is 3, which Numpy cheerfully tells us:

In [5]:
np.linalg.matrix_rank(A)

3

A good property to remember is that $\text{rank}(A) \le \min\{m, n\}$. As soon as we add a new row, we are guaranteed that it will be a linear combination of the previous rows:

In [6]:
# Row 3 = (Row 1) * 2 + (Row 2) + (Row 3)

B = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1],
              [2, 1, 1]])

np.linalg.matrix_rank(B)

3

In [7]:
# Transposing a matrix (flipping rows and columns) preserves rank
np.linalg.matrix_rank(B.T)

3

When $\text{rank}(A) = \min\{m, n\}$, we say that $A$ is **full rank**. 

**Question:** what is the rank of the matrix $A$ with columns $[a_1 \; a_2 \; a_3]$, shown below?

<img src='img/independence.png' style='height: 200px'>

First, notice that $A \in \mathbb{R}^{2 \times 3}$. Even though the set of vectors is not independent, the rank is still full (2) because at least 2 of the vectors are not colinear.

#### Application: Compression + Efficiency

As soon as there is some notion of redundant information, one can start thinking about applications in compression. Take the highly contrived but illustrative example of storing a matrix with 1,000,000 rows and 2 columns, where the 2nd column is exactly double the first column. That is,

<center>
$
A =
  \begin{bmatrix}
    a_{1,1} & 2a_{1,1} \\
    a_{2,1} & 2a_{2,1} \\
    \vdots \\
    a_{n,1} & 2a_{n,1} \\
  \end{bmatrix}
$
</center>

This would be a silly thing to do, when one could immediately save half of the memory by storing only the first column, and remembering to double the value whenever we need to index into the 2nd column. We can achieve this by representing $A$ as the following:

<center>
$
A =
  \begin{bmatrix}
    a_{1,1} & 2a_{1,1} \\
    a_{2,1} & 2a_{2,1} \\
    \vdots \\
    a_{n,1} & 2a_{n,1} \\
  \end{bmatrix}
=
  \begin{bmatrix}
    a_{1,1} \\
    a_{2,1} \\
    \vdots \\
    a_{n,1} \\
  \end{bmatrix}
  \begin{bmatrix}
  1 & 2
  \end{bmatrix}
$
</center>

This idea generalizes to matrices of any shape:

> A matrix $A$ with rank $r$ can be factored into matrices $Q \in \mathbb{R}^{m \times r}$ and $R \in \mathbb{R}^{r \times n}$

<img src='img\qr_factorization.png' style='height: 200px'>

Our lovely Mathematician friends have given us multiple algorithms that will produce these smaller matrices for us (see [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition)). We'll see one such algorithm in a later chapter.

Armed with this fact, we can quickly calculate how many numbers we have to store for the original matrix versus the factorized matrices:

$\;\;$ Size of $A = mn$

$\;\;$ Size of $Q$ + Size of $R = mr + rn = r(m+n)$

As soon as $r$ becomes appreciably smaller than $m$ and $n$, this trick can save a lot of space:

In [8]:
m = 1000
n = 1000
r = 10

# Storage cost of A
m*n

1000000

In [9]:
# Storage cost of QR
m*r + r*n

20000

**Multiplication Efficiency**

Amusingly, the math is identical when you estimate how many floating-point operations (FLOPs) it takes to compute $Ax = Q(Rx)$

$\;\;$ FLOPs to compute $Ax$ = (# rows) x FLOPs to compute ($\tilde{a}^\mathsf{T}x)$ $\sim O(mn)$

$\;\;$ FLOPs to compute $QRx\sim O(mr + rn) = O(r(m+n))$

### Nullspace

"The Nullspace" is a term that haunted me from my first linear algebra class. I remember thinking it sounded cool, but I had no grasp of what it really meant. I think the simplest way of understanding the idea is this: every matrix has a nullspace, which is a list of vectors. If you multiply your matrix $A$ by a vector $z$ and the result is the zero vector, then you put $z$ in the list.

> The nullspace of $A$ is the set of vectors, $z$, such that $Az = 0$

If we were to express this in code, `A.nullspace()` would return a list of vectors. This list would be infinitely long, but one can return a list of vectors that can be used to reconstruct any vector in the nullspace via linear combinations (we would call this finite set a **basis** for the nullspace). There isn't a direct way to return the nullspace of $A$ with Numpy, but we will cover how to do this in a later chapter (see Ch 4 - Singular Value Decomposition).

The **key idea** about vectors in the nullspace of $A$ is that they represent redundancy or flexibility in the system:

$A(x + z) = Ax + Az = Ax + 0 = Ax$

We can add any vector from the nullspace "for free." Visually, if we plot the points $x$ where $Ax$ equals some constant $b$, then any vector that is parallel to the line is in the nullspace of $A$:

<img src='img\nullspace.png' style='height: 200px'>

This is great if, for instance, we have $y=Ax$ where $y$ is a target we are trying to achieve. Lots of vectors in the nullspace means we have options to choose from for our input, and we can optimize on some other criteria.

However, if you recall our transmitter/receiver analogy from above, having vectors in the nullspace of our matrix can be a very bad thing! Imagine you are trying to reconstruct what your transmitters were sending from the sensor measurements you detect. If there are multiple ways to transmit signals that produce the same received measurements, **you can never know which signals were sent.**

### Singularity

This last idea, that you can't "undo" the transformation from multiplying $x$ by $A$, is mathematically expressed by saying that $A$ is not invertible. Precisely, there is no matrix $B$ such that 

<center>
    $x = B(Ax)$
</center>

This idea is also expressed by saying that "$A$ has no left inverse". In the special case where $A$ is square, you will encounter people using the term **singular** to mean a matrix which has no inverse. As best I can find, the term singular refers to an (incorrect) understanding that most square matrices are invertible, and so non-invertible matrices are special. 

### The Determinant

Another abstract concept you would undoubtedly find in a beginning linear algebra class is **the determinant** of a matrix. You probably were drilled at length on using [Cramer's Rule](https://en.wikipedia.org/wiki/Cramer%27s_rule) to calculate the determinant, and that was probably all you would have done with it.

The determinant is connected to all of the previous topics in this section, which I have grouped together because they all indicate whether or not a matrix operation can be undone. In applied terms, whether or not an estimation problem has a unique solution, or if a design problem has multiple choices.

The determinant is a number that **determines** if a square matrix is singular. That's where the name comes from. If $\det(A)=0$, then $A$ has no inverse.

Ok great. Why?

There is a great visual interpretation - we'll work in a 3-dimensional space so that we can draw it, but this extends to any number of dimensions. Suppose we have a cube with volume 1. Applying the matrix $A$ to the vectors which define the points on the boundary of the cube results in a parallelopiped with volume equal to the determinant of $A$:

<img src='img/determinant_1.png' style='height: 200px'>

If the determinant of $A$ is 0, the resulting parallelopiped has volume 0, which implies at least one dimension of the cube is flattened:

<img src='img/determinant_2.png' style='height: 200px'>

In the picture above, any information in the vertical direction is lost - you can't unflatten the shape because you don't know how high to stretch!

We can demonstrate this volume concept in code:

In [10]:
# Define a 3d unit cube
C = np.eye(3)
C

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [11]:
# Define a transformation matrix
A = np.random.randint(low=1, high=5, size=(3, 3))
A

array([[1, 4, 2],
       [1, 4, 4],
       [4, 4, 2]])

In [12]:
# Since the unit cube vectors correspond to an Identity matrix, the output of the transform
# is the same as the transformation matrix
A @ C

array([[1., 4., 2.],
       [1., 4., 4.],
       [4., 4., 2.]])

In [13]:
# The volume of the transformed cube is given by the determinant
np.linalg.det(A)

24.000000000000004

In [14]:
# If we make two of the columns of A identical, this will have the effect of 
# Making the parallelopiped effectively 2-dimensional. The determinant will be
# zero to reflect this.
A_flat = A
A_flat[:, 2] = A[:, 1]
np.linalg.det(A_flat)

0.0

### Unification

You may have noticed just now that, to create a matrix with a zero determinant, I made one column of A equal to another column. This dropped the rank of $A$. This was not a coincidence. In fact, the rank of $A$ can also be interpreted as the number of dimensions in the accessible output space (the **range**) of $A$ - if this number is less than the number of dimensions of $x$, then some kind of geometric flattening is going to happen and the determinant will be zero. 

At the same time, removing a dimension from the range of $A$ **adds** a dimension to the nullspace of $A$! There is a handy property:

> $\text{Rank}(A) + \text{dim Null}(A) = n$

All of the topics in this section are intimately connected, each communicating whether or not a matrix is invertible. To summarize, the following statements are equivalent:

- $A$ is full rank
- $A$ has only the zero vector in its nullspace
- the determinant of $A$ is non-zero
- $A$ has a left inverse